In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import re
import numpy as np


In [3]:
path = './result_241201'
file_list = os.listdir(path)

In [ ]:
tmp = pd.DataFrame()
df_eval = pd.DataFrame()
def sc_calc_acc_condition_with_temp(llm_model, few_shot_n, test_n, q_src_yn, p_ver, temp):
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
    opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}_{temp}.csv')]
    df = pd.DataFrame()
    if len(opt_file)>0 : 
        for f in opt_file:
            print(f)
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            print(acc)
            # print(df_eval)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [5]:
sc_calc_acc_condition_with_temp('l', 2, 10, 'Y','sys_prompt4', 0.2)

sc_l_result_2_10_Y_1_sys_prompt4_0.2.csv
80.0
         id gold o_result  cnt  equal_yn
0  74727945    0        0    5         1
1  74793352    0        0    5         1
2  75057003    1        1    5         1
3  75198212    2        1    5         0
4  76115814    1        0    4         0
5  76675433    2        2    5         1
6  76917593    0        0    4         1
7  77019129    1        1    5         1
8  77020489    1        1    5         1
9  77467788    1        1    5         1
sc_l_result_2_10_Y_4_sys_prompt4_0.2.csv
80.0
         id gold o_result  cnt  equal_yn
0  74693871    1        1    3         1
1  74936129    1        0    5         0
2  75266730    1        1    3         1
3  75492157    1        1    5         1
4  76045819    1        0    3         0
5  76774917    0        0    5         1
6  76784524    1        1    4         1
7  77068633    1        1    5         1
8  77370805    1        1    5         1
9  77543317    2        2    5         1
sc_l_r